In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.formula.api import ols
categorical_var = ["university", "sex", "gender_identity", "sexual_orientation", "ethnic_group", 'marital_status', 'student_accommodation', "work", 'RENDA_categorias', 'student_of', 'sedentary_behavior', "sedentary_2", "phq9", "phq9_severity", "gad7", "gad7_severity"]
db = pd.read_excel("BANCO_LIMPO_INT.xlsx")
#for var in categorical_var:
#    print(f"Response counts for {var}:")
#    print(db[var].value_counts())
#    print() 

creating contingency tables for every variable: 

In [11]:
xls = pd.ExcelFile('cat-tot-sheet.xlsx')
data_expanded = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet_name)
    expanded_df = df.loc[df.index.repeat(df['count'])].drop(columns=['count']).reset_index(drop=True)
    data_expanded[sheet_name] = expanded_df
with pd.ExcelWriter('contingency_tables.xlsx') as writer:
    for var1 in xls.sheet_names:
        for var2 in xls.sheet_names:
            if var1 != var2:
                merged = pd.concat([data_expanded[var1], data_expanded[var2]], axis=1)
                contingency_table = pd.crosstab(merged.iloc[:, 0], merged.iloc[:, 1], margins=True, margins_name='Total')
                contingency_table_df = contingency_table.reset_index()
                sheet_name = f'{var1}_vs_{var2}'[:30] 
                contingency_table_df.to_excel(writer, sheet_name=sheet_name, index=False)


testing for chi-squared and cramer's v:

In [ ]:
xlsx_file = pd.ExcelFile('contingency_tables.xlsx')
for sheet_name in xlsx_file.sheet_names:
    contingency_table = pd.read_excel(xlsx_file, sheet_name = sheet_name, index_col = 0)
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    n = contingency_table.to_numpy().sum()
    v_cramer = np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))
    chi_squared_results = pd.DataFrame({
                    'Estatística': ['Chi-quadrado', 'valor p', 'Graus de liberdade', "V de Cramer"],
                    'Valor': [chi2, p, dof, v_cramer]
                })
    print(" ")
    print(sheet_name)
    print(chi_squared_results)
    print(" ")
    if p < 0.05: 
        print("Hipótese nula rejeitada; há dependência das variáveis") 
        if v_cramer < 0.3: print("Dependência fraca entre as variáveis; ")
        elif v_cramer > 0.6: print("Dependência forte entre variáveis")
        else: print("Dependência razoável entre variáveis")
    else: print("Hipótese alternativa rejeitada; variáveis independentes")
